In [4]:
data='../data/'
# data/chip contains chip-seq data
# data/e contains mouse ESC HMR track
# data/mm10-blacklist.v2.bed.gz = mouse black list

#Loading packages and functions for finding PRC2 binding in LMRs in Mouse

import pandas as pd
import pyBigWig 

#please refer to s.table 1 to download the following bigwig files into the data/chip folder to run 
mez1=pyBigWig.open(data+'chip/SRX2528911.bw') # EZH2 rep1 mesc chip-atlas
mez2=pyBigWig.open(data+'chip/SRX2528912.bw') # EZH2 rep2 mesc chip-atlas
msu=pyBigWig.open(data+'chip/SRX4338248.bw') # SUZ12 mesc chip-atlas

#remove ENCODE blacklist regions - bias high signal regions that are problematic in ChIP experiments
def bl(t,b):
    bl=pd.read_table(data+b,header=None)
    bl['r']=bl.apply(lambda x: set(range(x[1],x[2])),axis=1)
    bg=bl.groupby(0)['r'].apply(list)
    bg=bg.apply(lambda x:set.union(*x))   
    t['r']=t.apply(lambda x: set(range(x[1],x[2])),axis=1)
    t['bl']=t.apply(lambda x: len(x['r'].intersection(bg[x['ch']])),axis=1)
    t=t[t['bl']==0].drop('r',axis=1)
    return t

In [7]:
# get prc2 binding in mouse ESC LMRs
folder='e';cell='em';
t=pd.read_csv(data+'meth/'+folder+'/'+cell+'.hmr',sep='\t',header=None)
t=t[~t[0].isin(['chrX','chrY','chrM'])].copy()
t=t[~t[0].str.contains('random')].copy()
t=t[~t[0].str.contains('Un')].copy()
t['d']=t[2]-t[1]
t.columns=['ch','b','e','d']
t=bl(t,'mm10-blacklist.v2.bed.gz')
t['ez1']=t.apply(lambda x:mez1.stats(x[0],x[1],x[2])[0],axis=1)
t['ez2']=t.apply(lambda x:mez2.stats(x[0],x[1],x[2])[0],axis=1)    
t['su']=t.apply(lambda x:msu.stats(x[0],x[1],x[2])[0],axis=1)
for c in ['ez1','ez2','su']:
    t[c]=t[c]/t[c].max()
t['pr']=t[['ez1','ez2','su']].mean(1)   
print(len(t))    

/local/scratch/moqri/slrmtmp.43007249/ipykernel_52922/3869699129.py:20: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  t['r']=t.apply(lambda x: set(range(x[1],x[2])),axis=1)
/local/scratch/moqri/slrmtmp.43007249/ipykernel_52922/1936297017.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  t['ez1']=t.apply(lambda x:mez1.stats(x[0],x[1],x[2])[0],axis=1)
/local/scratch/moqri/slrmtmp.43007249/ipykernel_52922/1936297017.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a valu

37919


In [9]:
# sort and save 
h=t.copy()
ht=h.sort_values('pr',ascending=False).head(1000)
ht[ht.d>100].to_csv('results/mm10_top_PRC2_d100.csv',index=False)